In [1]:
import pandas as pd
import numpy as np
from numpy.random import seed
seed(2)

df = pd.read_pickle(r'./combined_data_feb2023.pkl')

In [2]:
features = ['id','Ward Glucose', 'Haemoglobin',
       'Mean cell volume, blood', 'White blood cell count, blood',
       'Haematocrit', 'Platelets', 'Urea level, blood', 'Creatinine', 'Sodium',
       'Potassium', 'Lymphocytes', 'Neutrophils', 'C-Reactive Protein',
       'Eosinophils', 'Alkaline Phosphatase', 'Albumin',
       'Alanine Transaminase', 'Bilirubin', 'Total Protein',
       'Fibrinogen (clauss)', 'Glucose POCT Strip Blood', 'Ferritin',
       'D-Dimer', 'Ward Lactate', 'age', 'sex', 'SARS CoV-2 RNA','pathogenic']



In [3]:
#Manual split of training, validation and test set, and displays of their properties

start_date = '2014-1-1'
train_date = '2020-12-1'
val_date = '2021-03-1'

train = df[(df['date_culture']>=start_date)&(df['date_culture']<train_date)]

val = df[(df['date_culture']>=train_date)&(df['date_culture']<val_date)]

test = df[df['date_culture']>=val_date]

display('Train')
display(train['pathogenic'].value_counts(normalize=True))
display(len(train.groupby(['id','date_culture'])))

display('Val')
display(val['pathogenic'].value_counts(normalize=True))
display(len(val.groupby(['id','date_culture'])))

display('Test')
display(test['pathogenic'].value_counts(normalize=True))
display(len(test.groupby(['id','date_culture'])))

'Train'
pathogenic
0.0    0.761944
1.0    0.238056
Name: proportion, dtype: float64

13354
'Val'
pathogenic
0.0    0.885361
1.0    0.114639
Name: proportion, dtype: float64
1858
'Test'
pathogenic
0.0    0.915928
1.0    0.084072
Name: proportion, dtype: float64

5638

In [4]:
train_limited = train[features]
val_limited = val[features]
test_limited = test[features]

In [5]:
# Functions to feature engineer for baseline

def aggregate(df):
    summarise = df.groupby('id').agg({
                                   
                                        'Ward Lactate':['min','mean','max'], 
                                        'Ward Glucose':['min','mean','max'], 
                                        'Haemoglobin':['min','mean','max'], 
                                        'Mean cell volume, blood':['min','mean','max'], 
                                        'White blood cell count, blood':['min','mean','max'], 
                                        'Haematocrit':['min','mean','max'], 
                                        'Platelets':['min','mean','max'], 
                                        'Urea level, blood':['min','mean','max'], 
                                        'Creatinine':['min','mean','max'], 
                                        'Sodium':['min','mean','max'], 
                                        'Potassium':['min','mean','max'], 
                                        'Lymphocytes':['min','mean','max'], 
                                        'Neutrophils':['min','mean','max'], 
                                        'C-Reactive Protein':['min','mean','max'], 
                                        'Eosinophils':['min','mean','max'], 
                                        'Alkaline Phosphatase':['min','mean','max'], 
                                        'Albumin':['min','mean','max'], 
                                        'Alanine Transaminase':['min','mean','max'], 
                                        'Bilirubin':['min','mean','max'], 
                                        'Total Protein':['min','mean','max'], 
                                        'Fibrinogen (clauss)':['min','mean','max'], 
                                        'Glucose POCT Strip Blood':['min','mean','max'],  
                                        'Ferritin':['min','mean','max'], 
                                        'D-Dimer':['min','mean','max'], 
                                        'sex':'first',
                                        'age':'first',
                                        'pathogenic':'first'})


    return summarise

In [6]:
def aggregate_mean(df):
    summarise = df.groupby('id').agg({
                                   
                                        'Ward Lactate':'mean', 
                                        'Ward Glucose':'mean', 
                                        'Haemoglobin':'mean', 
                                        'Mean cell volume, blood':'mean', 
                                        'White blood cell count, blood':'mean', 
                                        'Haematocrit':'mean', 
                                        'Platelets':'mean', 
                                        'Urea level, blood':'mean', 
                                        'Creatinine':'mean', 
                                        'Sodium':'mean', 
                                        'Potassium':'mean', 
                                        'Lymphocytes':'mean', 
                                        'Neutrophils':'mean', 
                                        'C-Reactive Protein':'mean', 
                                        'Eosinophils':'mean', 
                                        'Alkaline Phosphatase':'mean', 
                                        'Albumin':'mean', 
                                        'Alanine Transaminase':'mean', 
                                        'Bilirubin':'mean', 
                                        'Total Protein':'mean', 
                                        'Fibrinogen (clauss)':'mean', 
                                        'Glucose POCT Strip Blood':'mean',  
                                        'Ferritin':'mean', 
                                        'D-Dimer':'mean', 
                                        'sex':'first',
                                        'age':'first',
                                        'pathogenic':'first'})


    return summarise

In [7]:
def aggregate_last(df):
    summarise = df.groupby('id').agg({
                                   
                                        'Ward Lactate':'last', 
                                        'Ward Glucose':'last', 
                                        'Haemoglobin':'last', 
                                        'Mean cell volume, blood':'last', 
                                        'White blood cell count, blood':'last', 
                                        'Haematocrit':'last', 
                                        'Platelets':'last', 
                                        'Urea level, blood':'last', 
                                        'Creatinine':'last', 
                                        'Sodium':'last', 
                                        'Potassium':'last', 
                                        'Lymphocytes':'last', 
                                        'Neutrophils':'last', 
                                        'C-Reactive Protein':'last', 
                                        'Eosinophils':'last', 
                                        'Alkaline Phosphatase':'last', 
                                        'Albumin':'last', 
                                        'Alanine Transaminase':'last', 
                                        'Bilirubin':'last', 
                                        'Total Protein':'last', 
                                        'Fibrinogen (clauss)':'last', 
                                        'Glucose POCT Strip Blood':'last',  
                                        'Ferritin':'last', 
                                        'D-Dimer':'last', 
                                        'sex':'first',
                                        'age':'first',
                                        'pathogenic':'first'})


    return summarise

In [8]:
# Three different approaches 
# i) Summarised approach using (min mean max)
# ii) Mean only
# iii) Last value (on the day of blood culture acquisition)

#Summarised
train_aggregated = aggregate(train_limited)
val_aggregated = aggregate(val_limited)
test_aggregated = aggregate(test_limited)

#Mean only
train_aggregated_mean = aggregate_mean(train_limited)
val_aggregated_mean = aggregate_mean(val_limited)
test_aggregated_mean = aggregate_mean(test_limited)

#Most recent value
train_aggregated_last = aggregate_last(train_limited)
val_aggregated_last = aggregate_last(val_limited)
test_aggregated_last = aggregate_last(test_limited)

In [9]:
train_combined = pd.concat([train_aggregated,val_aggregated],axis=0)
train_combined_mean = pd.concat([train_aggregated_mean,val_aggregated_mean],axis=0)
train_combined_last = pd.concat([train_aggregated_last,val_aggregated_last],axis=0)

In [10]:
train_combined.columns = ["_".join(pair) for pair in train_combined.columns]
test_aggregated.columns = ["_".join(pair) for pair in test_aggregated.columns]
train_combined = train_combined.reset_index()
train_combined_mean = train_combined_mean.reset_index()
train_combined_last = train_combined_last.reset_index()

test_aggregated = test_aggregated.reset_index()
test_aggregated_mean = test_aggregated_mean.reset_index()
test_aggregated_last = test_aggregated_last.reset_index()

In [11]:
display(train_combined['id'].nunique())
display(test_aggregated['id'].nunique())

15212
5638

In [12]:
# Median imputation fitted on train and applied to test

from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='median')


train_imputed = pd.DataFrame(si.fit_transform(train_combined),columns = si.get_feature_names_out())
test_imputed = pd.DataFrame(si.transform(test_aggregated),columns = si.get_feature_names_out())

train_combined_mean_imputed = pd.DataFrame(si.fit_transform(train_combined_mean),columns = si.get_feature_names_out())
test_imputed_mean = pd.DataFrame(si.transform(test_aggregated_mean),columns = si.get_feature_names_out())

train_combined_last_imputed = pd.DataFrame(si.fit_transform(train_combined_last),columns = si.get_feature_names_out())
test_imputed_last = pd.DataFrame(si.transform(test_aggregated_last),columns = si.get_feature_names_out())

In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs')

data = train_imputed.iloc[:,2:]
test = test_imputed.iloc[:,2:]

X_train = data.iloc[:,:-1]
y_train = data.iloc[:,-1]
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

data_mean = train_combined_mean_imputed.iloc[:,2:]
test_mean = test_imputed_mean.iloc[:,2:]

X_train_mean = data_mean.iloc[:,:-1]
y_train_mean = data_mean.iloc[:,-1]
X_test_mean = test_mean.iloc[:,:-1]
y_test_mean = test_mean.iloc[:,-1]

data_last = train_combined_last_imputed.iloc[:,2:]
test_last = test_imputed_last.iloc[:,2:]

X_train_last = data_last.iloc[:,:-1]
y_train_last = data_last.iloc[:,-1]
X_test_last = test_last.iloc[:,:-1]
y_test_last = test_last.iloc[:,-1]

In [14]:
model_base = lr.fit(X_train,y_train)
predictions = model_base.predict_proba(X_test)[:,1]

model_mean = lr.fit(X_train_mean,y_train_mean)
predictions_mean = model_mean.predict_proba(X_test_mean)[:,1]

model_last = lr.fit(X_train_last,y_train_last)
predictions_last = model_last.predict_proba(X_test_last)[:,1]

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

In [15]:
from functions.tools import metrics

In [16]:
print('Summarised values')
display(metrics(predictions,y_test))

Summarised values
Best Threshold=0.200951
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96      5164
         1.0       0.51      0.59      0.55       474

    accuracy                           0.92      5638
   macro avg       0.74      0.77      0.75      5638
weighted avg       0.92      0.92      0.92      5638

Specificity 0.9486831913245546
Sensitivity 0.5864978902953587
ROC score 0.7346515310474659
AP 0.47143389720507356
PPV 0.5119705340699816
NPV 0.9615309126594701
BS 0.06505960806079497
AUPRC 0.47143389720507356


None

In [17]:
print('Mean only')
display(metrics(predictions_mean, y_test_mean))

Mean only
Best Threshold=0.277253
              precision    recall  f1-score   support

         0.0       0.96      0.97      0.96      5164
         1.0       0.59      0.51      0.55       474

    accuracy                           0.93      5638
   macro avg       0.77      0.74      0.75      5638
weighted avg       0.92      0.93      0.93      5638

Specificity 0.9680480247869868
Sensitivity 0.5063291139240507
ROC score 0.7227981285563475
AP 0.4493867220531532
PPV 0.5925925925925926
NPV 0.9552837760366902
BS 0.06883521859099855
AUPRC 0.4493867220531532


None

In [18]:
print('Last values')
display(metrics(predictions_last,y_test_last))

Last values
Best Threshold=0.238938
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96      5164
         1.0       0.54      0.58      0.56       474

    accuracy                           0.92      5638
   macro avg       0.75      0.77      0.76      5638
weighted avg       0.93      0.92      0.92      5638

Specificity 0.9552672347017815
Sensitivity 0.5759493670886076
ROC score 0.7457323829040388
AP 0.4765332724345627
PPV 0.5416666666666666
NPV 0.960849240358395
BS 0.06614090718321455
AUPRC 0.4765332724345627


None

In [19]:
# 5-fold cross validation of scores

from sklearn.model_selection import cross_val_score

score = cross_val_score(model_base, X_train, y_train, cv=5, scoring='roc_auc')
score_mean = cross_val_score(model_mean, X_train_mean, y_train_mean, cv=5, scoring='roc_auc')
score_last = cross_val_score(model_last, X_train_last, y_train_last, cv=5, scoring='roc_auc')

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

In [23]:
print('AUROC scores from 5-fold CV')
display(score)
display(score_mean)
display(score_last)

# Scores with inter-quartile ranges

print(np.median(score), np.quantile(score,0.25), np.quantile(score,0.75))

print(np.median(score_mean),np.quantile(score_mean,0.25),np.quantile(score_mean,0.75))

print(np.median(score_last),np.quantile(score_last,0.25),np.quantile(score_last,0.75))

AUROC scores from 5-fold CV


0.47143389720507356
array([0.73028727, 0.8364782 , 0.8288315 , 0.68942633, 0.63515884])
array([0.75127527, 0.85330024, 0.8178463 , 0.68467399, 0.63850674])

0.47143389720507356 0.47143389720507356 0.47143389720507356
0.7302872742587014 0.6894263260610236 0.8288315015298304
0.7512752654570647 0.6846739938806783 0.8178462957139792


In [24]:
# 5-fold cross validation of scores

from sklearn.model_selection import cross_val_score

score = cross_val_score(model_base, X_train, y_train, cv=5, scoring='average_precision')
score_mean = cross_val_score(model_mean, X_train_mean, y_train_mean, cv=5, scoring='average_precision')
score_last = cross_val_score(model_last, X_train_last, y_train_last, cv=5, scoring='average_precision')

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

In [25]:
#AUPRC scores

print('AURPC scores from 5-fold CV')
display(score)
display(score_mean)
display(score_last)

# Scores with inter-quartile ranges

print(np.median(score), np.quantile(score,0.25), np.quantile(score,0.75))
print(np.median(score_mean),np.quantile(score_mean,0.25),np.quantile(score_mean,0.75))
print(np.median(score_last),np.quantile(score_last,0.25),np.quantile(score_last,0.75))


AURPC scores from 5-fold CV


array([0.55048453, 0.84547948, 0.85033946, 0.45342842, 0.4358811 ])
array([0.54432863, 0.77945724, 0.75861822, 0.4284046 , 0.39496819])
array([0.57656793, 0.8131919 , 0.77134434, 0.45680136, 0.43487268])

0.5504845340692617 0.45342842164994235 0.8454794787696785
0.5443286298770807 0.4284045975883137 0.7586182188075594
0.5765679343030642 0.4568013634175667 0.7713443419293886


In [22]:
#This bootstraps the hold out set 2000 fold to derive the 95% confidence intervals for each of the approaches:

from functions.bootstrap import *

print('last_value')
score, ci_lower, ci_upper, scores = score_ci(y_test_last, predictions_last, score_fun=rc)
print('auroc')
print(round(score,2),'(',round(ci_lower,2),'-',round(ci_upper,2),')')
score, ci_lower, ci_upper, scores = score_ci(y_test_last, predictions_last, score_fun=ap)
print('auprc')
print(round(score,2),'(',round(ci_lower,2),'-',round(ci_upper,2),')')
print('')
print('mean_value')
score, ci_lower, ci_upper, scores = score_ci(y_test_mean, predictions_mean, score_fun=rc)
print('auroc')
print(round(score,2),'(',round(ci_lower,2),'-',round(ci_upper,2),')')
score, ci_lower, ci_upper, scores = score_ci(y_test_mean, predictions_mean, score_fun=ap)
print('auprc')
print(round(score,2),'(',round(ci_lower,2),'-',round(ci_upper,2),')')
print('')
print('summarised')
score, ci_lower, ci_upper, scores = score_ci(y_test, predictions, score_fun=rc)
print('auroc')
print(round(score,2),'(',round(ci_lower,2),'-',round(ci_upper,2),')')
score, ci_lower, ci_upper, scores = score_ci(y_test, predictions, score_fun=ap)
print('auprc')
print(round(score,2),'(',round(ci_lower,2),'-',round(ci_upper,2),')')

last_value
auroc
0.75 ( 0.71 - 0.78 )
auprc
0.48 ( 0.43 - 0.53 )

mean_value
auroc
0.72 ( 0.69 - 0.76 )
auprc
0.45 ( 0.4 - 0.5 )

summarised
auroc
0.73 ( 0.7 - 0.76 )
auprc
0.47 ( 0.42 - 0.52 )
